# Exercícios – Structured Streaming com Kafka

### Ler o tópico do kafka “topic-kvspark” em modo batch

In [ ]:
from pyspark.sql.functions import *

In [ ]:
topic_read = spark.read\
    .format("kafka")\
    .option("kafka.bootstrap.servers", "kafka:9092")\
    .option("subscribe", "topic-kvspark")\
    .load()

### Visualizar o schema do tópico

In [ ]:
topic_read.printSchema()

### Visualizar o tópico com o campo key e value convertidos em string

In [ ]:
# visualizando os dados dos campos key e value no formato binário:
topic_binary = topic_read.select(col("key"), col("value"))
topic_binary.show(10)

In [ ]:
# utilizando função 'cast' para converter os campos em string:
topic_string = topic_read.select(col("key").cast("string"), col("value").cast("string"))
topic_string.show(10)

### Ler o tópico do kafka “topic-kvspark” em modo streaming

In [ ]:
topic_read_stream = spark.readStream\
    .format("kafka")\
    .option("kafka.bootstrap.servers", "kafka:9092")\
    .option("subscribe", "topic-kvspark")\
    .load()

### Visualizar o schema do tópico em streaming

In [ ]:
topic_read_stream.printSchema()

### Alterar o tópico em streaming com o campo key e value convertidos para string

In [ ]:
topic_string_stream = topic_read_stream.select(col("key").cast("string"), col("value").cast("string"))

### Salvar o tópico em streaming no tópico topic-kvspark-output a cada 5 segundos

In [ ]:
topic_string_stream_output = topic_string_stream.writeStream\
    .format("kafka")\
    .option("kafka.bootstrap.servers", "kafka:9092")\
    .option("topic", "topic-kvspark-output")\
    .option("checkpointLocation", "/user/thiago/kafka_checkpoint")\
    .trigger(continuous = '5 second')\
    .start()

In [ ]:
topic_string_stream_output.status